In [1]:
import numpy as np
import sys
import umap
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/zata/zippy/moorej3/miniforge3/envs/notebook/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-05 12:19:47.580051: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 12:19:48.493004: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def Run_UMAP(data, neighbors, latentDim, name):
    reducer = umap.UMAP(n_neighbors = neighbors)
    x = reducer.fit_transform(data)
    outputFile = open("UMAP."+str(neighbors)+"."+str(latentDim)+"."+name+".txt", "w+")
    for entry in x:
        print(entry[0], "\t", entry[1], file=outputFile)
    outputFile.close()

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [27]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            cce = tf.keras.losses.categorical_crossentropy(data, reconstruction)  
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(cce, axis=1))    

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            beta = 0.001
            total_loss = reconstruction_loss + beta * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [19]:
def One_Hot_Encode(seq):
    baseDict = {"A":0, "C":1, "G":2, "T":3}
    oneHotArray = []
    seq = seq.upper()
    seqArray = np.array(list(seq))
    for base in seqArray:
        code = [0, 0, 0, 0]
        if base != "N":
            code[baseDict[base]] = 1
        else:
            code = [0.25, 0.25, 0.25, 0.25]
        oneHotArray.append(code)
    return oneHotArray

In [20]:
def Process_Sequences(fasta, seqLength):
    elementList = [next(fasta).lstrip(">").rstrip()]
    inputData = []
    seq=''
    for line in fasta:
        if ">" not in line:
            seq += line.rstrip()
        else:
            inputData.append(One_Hot_Encode(seq))
            elementList.append(line.lstrip(">"))
            seq=''
    inputData.append(One_Hot_Encode(seq))
    inputData = np.array(inputData)
    inputData = inputData.reshape(inputData.shape[0], seqLength, 4)
    return elementList, inputData

In [21]:
latentDim = 10 #int(sys.argv[1])
seqLength = 300
fasta=open("/home/moorej3/Lab/ENCODE/Encyclopedia/V7/Registry/V7-hg38/CORVUS/Input-Sequences/K562.PLS-ELS.fa")
#fasta=open(sys.argv[2])
ccres, input = Process_Sequences(fasta, seqLength)
fasta.close()

In [28]:
latentDim = 10 #int(sys.argv[1])

encoder_inputs = keras.Input(shape=(seqLength, 4))
x = layers.Conv1D(filters=16, kernel_size=10, strides=1, activation="relu", padding="same")(encoder_inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latentDim, name="z_mean")(x)
z_log_var = layers.Dense(latentDim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
print(encoder.summary())

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 300, 4)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 300, 16)   │        656 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 150, 16)   │          0 │ conv1d_3[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 2400)      │          0 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 16)        │     38,416 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 10)        │        170 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 10)        │        170 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling_3          │ (None, 10)        │          0 │ z_mean[0][0],     │
│ (Sampling)          │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,412 (153.95 KB)

 Trainable params: 39,412 (153.95 KB)

 Non-trainable params: 0 (0.00 B)

None


In [29]:
latent_inputs = keras.Input(shape=(latentDim,))
x = layers.Dense(16, activation="relu")(latent_inputs)
x = layers.Dense(2400, activation="relu")(x)
x = layers.Reshape((150, 16))(x)
x = layers.UpSampling1D(size=2)(x)
x = layers.Conv1DTranspose(filters=16, kernel_size=10, activation="relu", strides=1, padding="same")(x)
decoder_outputs = layers.Conv1DTranspose(4, 3, activation="softmax", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
print(decoder.summary())

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2400)           │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 150, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_3 (UpSampling1D)  │ (None, 300, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_6              │ (None, 300, 16)        │         2,576 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_7              │ (None, 300, 4)         │           196 │
│ (Conv1DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,748 (170.89 KB)

 Trainable params: 43,748 (170.89 KB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [31]:
vae.fit(input, epochs=25)


Epoch 1/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 96s 29ms/step - kl_loss: 54.0184 - loss: 403.7706 - reconstruction_loss: 403.7168 
Epoch 2/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 93s 29ms/step - kl_loss: 57.4909 - loss: 397.9042 - reconstruction_loss: 397.8468 
Epoch 3/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 94s 29ms/step - kl_loss: 50.7111 - loss: 397.6177 - reconstruction_loss: 397.5670 
Epoch 4/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 94s 29ms/step - kl_loss: 48.1009 - loss: 397.3908 - reconstruction_loss: 397.3427 
Epoch 5/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 94s 29ms/step - kl_loss: 46.5429 - loss: 397.3634 - reconstruction_loss: 397.3165 
Epoch 6/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 94s 29ms/step - kl_loss: 44.8973 - loss: 397.3999 - reconstruction_loss: 397.3553 
Epoch 7/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 94s 29ms/step - kl_loss: 43.6615 - loss: 397.2509 - reconstruction_loss: 397.2072 
Epoch 8/25
3226/3226 ━━━━━━━━━━━━━━━━━━━━ 93s 29ms/step - kl_loss: 42.5436 - loss: 397.1806 - reconstruction_loss: 397.1383 


In [32]:
z_mean, _, _ = vae.encoder.predict(input)
Run_UMAP(z_mean, 100, latentDim, "K562-16CNN-16Dense-New3")

3226/3226 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step


/zata/zippy/moorej3/miniforge3/envs/notebook/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
